In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np

import nltk
import string
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
#from pattern.text.en import singularize

In [2]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\veselin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def clean(text):
    # Lowercase text
    text = text.lower()
    
    # Remove numbers
    #text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    text = text.replace(':', ' ')
    text = text.replace('\'', ' ')
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    # Remove whitespace from text
    text = " ".join(text.split())
    
    # Remove stopwords function
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    text = [word for word in word_tokens if word not in stop_words]
    
    # Lemmatize words
    #text = [lemmatizer.lemmatize(word, pos ='v') for word in text]
    
    # Stem words 
    #text = [stemmer.stem(word) for word in text]
    
    return text


def simple_clean(text):
    # Lowercase text
    text = text.lower()
    
    # Remove numbers
    #text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text


## START

In [5]:
df_sentences = pd.read_csv('sentences.csv')
df_biased_words = pd.read_excel("bias_words.xlsx")

In [6]:
to_exclude = ['she', 'her', 'hers', 'herself', 'he', 'himself', 'him', 'his']
df_biased_words = df_biased_words[~df_biased_words['Biased Words or Phrases'].isin(to_exclude)]
df_biased_words = df_biased_words.drop_duplicates()

In [7]:
male_words = (df_biased_words.loc[df_biased_words['Masculine/Feminine Bias'] == 'Masculine Bias'])['Biased Words or Phrases'].values
female_words = (df_biased_words.loc[df_biased_words['Masculine/Feminine Bias'] == 'Feminine Bias'])['Biased Words or Phrases'].values

In [31]:
# male_words = df_biased_words['male']
# male_words = male_words.dropna()
# male_words = [stemmer.stem(word) for word in male_words]
# female_words = df_biased_words['female']
# female_words = female_words.dropna()
# female_words = [stemmer.stem(word) for word in female_words]

In [33]:
male_words

['activ',
 'adventur',
 'aggress',
 'ambitio',
 'anali',
 'assert',
 'athlet',
 'autonom',
 'boast',
 'challeng',
 'compet',
 'confid',
 'courag',
 'decid',
 'decis',
 'decis',
 'determin',
 'domin',
 'domina',
 'forc',
 'greedi',
 'headstrong',
 'hierarch',
 'hostil',
 'implus',
 'independen',
 'individu',
 'intellect',
 'lead',
 'logic',
 'masculin',
 'object',
 'opinion',
 'outspoken',
 'persist',
 'principl',
 'reckless',
 'stubborn',
 'superior',
 'self-confiden',
 'self-sufficien',
 'self-relian',
 'manmad',
 'chairman',
 'son',
 'fireman',
 'freshman',
 'man',
 'mankind',
 'manpow',
 'boyfriend',
 'husband',
 'policeman',
 'walter',
 'brother',
 'spokesman',
 'upperclassman',
 'he',
 'him',
 'hi',
 'himself',
 'gentleman',
 'alumnu',
 'alumni',
 'man up',
 'mr.',
 'man-mad',
 'the common man',
 'mailman',
 'steward',
 'actor',
 'congressman']

In [34]:
female_words

['she',
 'affection',
 'child',
 'cheer',
 'commit',
 'commun',
 'compass',
 'connect',
 'consider',
 'cooperat',
 'depend',
 'emotiona',
 'empath',
 'feminin',
 'flatter',
 'gentl',
 'honest',
 'interperson',
 'interdependen',
 'interpersona',
 'kind',
 'kinship',
 'loyal',
 'modesti',
 'nag',
 'nurtur',
 'pleasant',
 'polit',
 'quiet',
 'respon',
 'sensitiv',
 'submiss',
 'support',
 'sympath',
 'tender',
 'togeth',
 'trust',
 'understand',
 'warm',
 'whin',
 'yield',
 'daughter',
 'wife',
 'girlfriend',
 'waitress',
 'sister',
 'her',
 'her',
 'herself',
 'ladi',
 'alumna',
 'alumna',
 'hyster',
 'shrill',
 'nag',
 'mrs.',
 'miss.',
 'ms.',
 'stewardess',
 'actress',
 'wife',
 'sister',
 'she',
 'her']

In [127]:
df_new = pd.DataFrame()

for index, sentence in df_sentences.iterrows():
    temp_sentence = sentence['sentences']
    tokenized_sentence = clean(temp_sentence)
    category = 'neutral'
    word_in_sentence = 'None'
    word = 'None'
    
    # check for male words
    for male_word in male_words:
        if re.search(r"\b{}\b".format(male_word), temp_sentence.lower().strip()):
            category = 'masculine'
            word_in_sentence = male_word
            word = male_word
        else:
            for token in tokenized_sentence:
                if len(male_word) > 3:

                    if simple_clean(male_word) == token[:len(male_word)]:
                        category = 'masculine'
                        word_in_sentence = token
                        word = male_word
                    elif simple_clean(male_word) == token[-len(male_word):]:
                        category = 'masculine'
                        word_in_sentence = token
                        word = male_word
            
    if category == 'masculine':
        dict = {'sentence': temp_sentence,
                'word_in_Sentence': word_in_sentence,
                'biased_term': word,
                'category': category
               }
        df_new = df_new.append(dict, ignore_index = True)
        
    # check for female words
    for female_word in female_words:
        if re.search(r"\b{}\b".format(female_word), temp_sentence.lower().strip()):
            category = 'feminine'
            word_in_sentence = female_word
            word = female_word
        else:
            for token in tokenized_sentence:
                if len(female_word) > 3:
                    if simple_clean(female_word) == token[:len(female_word)]:
                        category = 'feminine'
                        word_in_sentence = token
                        word = female_word
                    elif simple_clean(female_word) == token[-len(female_word):]:
                        category = 'feminine'
                        word_in_sentence = token
                        word = female_word
                    
    if category == 'feminine':
        dict = {'sentence': temp_sentence,
                'word_in_Sentence': word_in_sentence,
                'biased_term': word,
                'category': category
               }
        df_new = df_new.append(dict, ignore_index = True)
    
    if index%10000 == 0:
        print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000


In [38]:
df_new = pd.DataFrame()

for index, sentence in df_sentences.iterrows():
    temp_sentence = sentence['sentences']
    tokenized_sentence = clean(temp_sentence)
    words_in_sentence = []
    words = []
    
    if len(temp_sentence) < 180 and temp_sentence[0].isupper():
        # check for male words
        for male_word in male_words:
            if re.search(r"\b{}\b".format(male_word), temp_sentence.lower().strip()):
                words_in_sentence.append([male_word, 'M'])
                words.append([male_word, 'M'])
            else:
                for token in tokenized_sentence:
                    if len(male_word) > 3:
                        if simple_clean(male_word) == token[:len(male_word)]:
                            words_in_sentence.append([token, 'M'])
                            words.append([male_word, 'M'])
                        elif simple_clean(male_word) == token[-len(male_word):]:
                            if token[:len(male_word)] != token[-len(male_word):]:
                                words_in_sentence.append([token, 'M'])
                                words.append([male_word, 'M'])

        # check for female words
        for female_word in female_words:
            if re.search(r"\b{}\b".format(female_word), temp_sentence.lower().strip()):
                words_in_sentence.append([female_word, 'F'])
                words.append([male_word, 'F'])
            else:
                for token in tokenized_sentence:
                    if len(female_word) > 3:
                        if simple_clean(female_word) == token[:len(female_word)]:
                            words_in_sentence.append([token, 'F'])
                            words.append([female_word, 'F'])
                        elif simple_clean(female_word) == token[-len(female_word):]:
                            if token[:len(female_word)] != token[-len(female_word):]:
                                words_in_sentence.append([token, 'F'])
                                words.append([female_word, 'F'])

        if len(words) > 0:
            dict = {'sentence': temp_sentence,
                    'word_in_Sentence': words_in_sentence,
                    'biased_term': words}
            df_new = df_new.append(dict, ignore_index = True)

        if index%10000 == 0:
            print(index)

20000
30000
40000
60000
80000
90000
100000
120000
130000


In [30]:
df_new.to_csv('male_female_last_cleaned.csv', index=False)

In [28]:
df_new = pd.read_csv('male_female_last.csv')
df_new

,biased_term,sentence,word_in_Sentence
0,"[['active', 'M']]","Our client, located in Houston, is actively se...","[['actively', 'M']]"
1,"[['compet', 'M'], ['competitive', 'M']]","Privately held, we offer exceptional benefits,...","[['competitive', 'M'], ['competitive', 'M']]"
2,"[['respon', 'F']]","As part of an account team, you will be respon...","[['responsible', 'F']]"
3,"[['lead', 'M']]",Job Overview Apex is an environmental consult...,"[['leadership', 'M']]"
4,"[['respon', 'F']]",Driven by an entrepreneurial spirit and a dedi...,"[['responsive', 'F']]"
...,...,...,...
26107,"[['challeng', 'M']]","Ideally, you love the challenges of being a pr...","[['challenges', 'M']]"
26108,"[['individual', 'M']]","In addition, this individual will assist with ...","[['individual', 'M']]"
26109,"[['principle', 'M'], ['makes decisions easily'...",Candidate must also possess expertise in appli...,"[['principles', 'M'], ['support', 'F']]"
26110,"[['analy', 'M'], ['analytical', 'M']]",Strong analytical skills that will allow prope...,"[['analytical', 'M'], ['analytical', 'M']]"


In [29]:
def clean_sentence(sentence):
    temp = " ".join(sentence.split())
    temp = temp.strip()

    char_to_remove = 0
    for x in temp.split()[0]:
        if not x.isalpha():
            char_to_remove += 1
    temp = temp[char_to_remove: len(temp) - char_to_remove]
    return sentence

df_new['sentence'] = df_new.sentence.map(lambda x:clean_sentence(x))

### Explore filtered sentences

In [129]:
df_new.category.value_counts()

masculine    23828
feminine     23106
Name: category, dtype: int64

In [168]:
df_new.biased_term.value_counts().values

array([8332, 7085, 6953, 3067, 2627, 2355, 1402, 1214, 1069, 1022,  965,
        859,  809,  712,  681,  536,  519,  487,  460,  430,  407,  379,
        331,  326,  308,  286,  266,  250,  203,  195,  186,  167,  163,
        143,  109,  102,  101,   99,   98,   94,   90,   75,   75,   66,
         64,   58,   53,   51,   49,   44,   41,   37,   35,   33,   32,
         30,   30,   27,   25,   24,   23,   22,   22,   20,   19,   18,
          7,    7,    6,    6,    5,    5,    5,    5,    3,    3,    3,
          3,    3,    2,    2,    1,    1,    1,    1,    1,    1,    1,
          1,    1], dtype=int64)

In [173]:
not_found_words = []
found_words = df_new.biased_term.value_counts().keys()
for word in list(male_words) + list(female_words):
    if word not in found_words:
        not_found_words.append(word)

In [174]:
len(found_words)

90

In [175]:
len(not_found_words)

47

In [166]:
not_found_words

['aggress',
 'headstrong',
 'implusive',
 'masculine',
 'stubborn',
 'self-confiden',
 'self-sufficien',
 'self-relian',
 'manmade',
 'freshman',
 'mankind',
 'boyfriend',
 'policeman',
 'spokesman',
 'upperclassman',
 'alumnus',
 'man up',
 'Mr.',
 'mankind',
 'freshman',
 'man-made',
 'the common man',
 'mailman',
 'policeman',
 'congressman',
 'acts as a leader',
 'defends own beliefs',
 'forceful',
 'has leadership abilities',
 'individualistic',
 'makes decisions easily',
 'affectionate',
 'feminine',
 'flatterable',
 'interpersonal',
 'kinship',
 'modesty',
 'nag',
 'submissive',
 'whin',
 'girlfriend',
 'alumna',
 'alumnae',
 'hysterical',
 'shrill',
 'nagging',
 'Mrs.',
 'Ms.',
 'stewardess',
 'actress']

### Save data

In [144]:
to_save = pd.DataFrame(columns=df_new.columns)
for word in found_words:
    temp = df_new.loc[df_new.biased_term == word]
    temp = temp.head(6)
    frames = [to_save, temp]
    to_save = pd.concat(frames)

In [146]:
to_save.reset_index(inplace=True)

In [147]:
to_save = to_save.drop('index', axis=1)

In [148]:
to_save = to_save.sample(frac=1)

In [176]:
len(to_save)

468

In [105]:
df1, df2, df3, df4 = np.array_split(to_save, 4)

In [106]:
df2

,biased_term,sentence,word_in_Sentence,sort
7574,"[['confident', 'M']]",Maintain strict consumer confidentiality.,"[['confidentiality', 'M']]",confidentiality
21408,"[['confident', 'M']]",You will have your own list of clients to look...,"[['confident', 'M']]",confident
8928,"[['confident', 'M']]",You will be the first person new customers spe...,"[['confident', 'M']]",confident
8820,"[['confident', 'M'], ['lead', 'M']]",They will be confident at leading groups of de...,"[['confident', 'M'], ['leading', 'M']]",confident
8806,"[['confident', 'M']]",Consultative and confident approach to calls.,"[['confident', 'M']]",confident
...,...,...,...,...
20279,"[['challeng', 'M']]",To be successful in this role you will need to...,"[['challenging', 'M']]",challenging
17756,"[['challeng', 'M']]",But if you want a role that is fascinating and...,"[['challenging', 'M']]",challenging
24269,"[['makes decisions easily', 'F']]",She is looking forward to hiring someone who i...,"[['quiet', 'F']]",quiet
11648,"[['makes decisions easily', 'F']]",She is looking forward to hiring someone who i...,"[['quiet', 'F']]",quiet


In [32]:
df1.to_csv('df1.csv', index=False)
df2.to_csv('df2.csv', index=False)
df3.to_csv('df3.csv', index=False)
df4.to_csv('df4.csv', index=False)

In [2]:
df_new = pd.read_csv('male_female_last_cleaned.csv')

In [72]:
def return_biased_term(list_of_terms):
    list_of_terms = list_of_terms.replace('[', '')
    list_of_terms = list_of_terms.replace(']', '')
    list_of_terms = list_of_terms.replace('\'', '')
    return list_of_terms.split(',')[0]
df_new['sort'] = df_new.word_in_Sentence.map(lambda x: return_biased_term(x))

In [73]:
df_new = df_new.sort_values(by=['sort'])

In [100]:
# TO DO at least 4 at most 10 for each word, then divide into 4
to_save = pd.DataFrame(columns=df_new.columns)
for word in list(set(df_new.sort.values)):
    temp = df_new.loc[df_new.sort == word]
    temp = temp.head(5)
    if len(temp) > 4:
        frames = [to_save, temp]
        to_save = pd.concat(frames)

In [120]:
to_save = to_save.drop(['sort', 'biased_term'], axis=1)

In [121]:
df1, df2, df3, df4 = to_save.iloc[:245], to_save.iloc[245:245+245], to_save.iloc[245+245:245+245+245], to_save.iloc[245+245+245:]

In [123]:
df1.to_csv('df1_new.csv', index=False)
df2.to_csv('df2_new.csv', index=False)
df3.to_csv('df3_new.csv', index=False)
df4.to_csv('df4_new.csv', index=False)

In [126]:
df1

,sentence,word_in_Sentence
1003,Provide on-going reporting of instrument and a...,"[['analyzer', 'M']]"
1004,Arrange for contract services support for the ...,"[['analyzer', 'M'], ['support', 'F']]"
1005,Communicate employee concerns relating to inst...,"[['analyzer', 'M']]"
1006,Perform formal and informal audits of the inst...,"[['analyzer', 'M']]"
1002,"Develop, update, and track action plans pertai...","[['analyzer', 'M']]"
...,...,...
10201,Maintain confidentiality of all interactions.,"[['confidentiality', 'M']]"
15886,Ensuring patient confidentiality.,"[['confidentiality', 'M']]"
5312,To adhere to company policy and procedures in ...,"[['confidentiality', 'M']]"
5996,Maintain confidentiality of all interactions.,"[['confidentiality', 'M']]"


### END

In [36]:
df_new.to_csv('annotated2.csv', index = False)

In [3]:
df = pd.read_csv("annotated.csv")

In [6]:
text = "I'm in the responsibilities:the."
translator = str.maketrans(' ', ' ', string.punctuation)
text = text.translate(translator)

In [7]:
text

'Im in the responsibilitiesthe'

In [8]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
import re

text = "I'm in the responsibilities:the."
clean = re.sub(r"[,.;@#?!&$]+\ *", " ", text)

In [12]:
clean

"I'm in the responsibilities:the "

In [15]:
text.replace(':', ' ')
text.replace('\'', ' ')

'I m in the responsibilities:the.'

In [26]:
male_words

['activ',
 'adventur',
 'aggress',
 'ambitio',
 'anali',
 'assert',
 'athlet',
 'autonom',
 'boast',
 'challeng',
 'compet',
 'confid',
 'courag',
 'decid',
 'decis',
 'decis',
 'determin',
 'domin',
 'domina',
 'forc',
 'greedi',
 'headstrong',
 'hierarch',
 'hostil',
 'implus',
 'independen',
 'individu',
 'intellect',
 'lead',
 'logic',
 'masculin',
 'object',
 'opinion',
 'outspoken',
 'persist',
 'principl',
 'reckless',
 'stubborn',
 'superior',
 'self-confiden',
 'self-sufficien',
 'self-relian',
 'manmad',
 'chairman',
 'son',
 'fireman',
 'freshman',
 'man',
 'mankind',
 'manpow',
 'boyfriend',
 'husband',
 'policeman',
 'walter',
 'brother',
 'spokesman',
 'upperclassman',
 'he',
 'him',
 'hi',
 'himself',
 'gentleman',
 'alumnu',
 'alumni',
 'man up',
 'mr.',
 'man-mad',
 'the common man',
 'mailman',
 'steward',
 'actor',
 'congressman']

In [43]:
df_new.values[0][2].split()

['Food52,',
 'a',
 'fast-growing,',
 'James',
 'Beard',
 'Award-winning',
 'online',
 'food',
 'community',
 'and',
 'crowd-sourced',
 'and',
 'curated',
 'recipe',
 'hub,',
 'is',
 'currently',
 'interviewing',
 'full-',
 'and',
 'part-time',
 'unpaid',
 'interns',
 'to',
 'work',
 'in',
 'a',
 'small',
 'team',
 'of',
 'editors,',
 'executives,',
 'and',
 'developers',
 'in',
 'its',
 'New',
 'York',
 'City',
 'headquarters.']

In [88]:
he = 0
she = 0
neutral = 0
data_to_label = []

for x in df_new['sentence']:
    temp = x.lower().split()
    male = False
    female = False
    for y in temp:
        if "he" == y:
            male = True
    for z in temp:
        if "she" == z:
            female = True
    if male and female:
        neutral +=1
        data_to_label.append([x, 'neutral'])
    elif male:
        data_to_label.append([x, 'generic_he'])
        he +=1
    elif female:
        data_to_label.append([x, 'generic_she'])
        she +=1
         

In [89]:
df_generic_she_generic_he = pd.DataFrame(data_to_label)
df_generic_she_generic_he.columns = ['sentence', 'label']

In [90]:
df_cleaned = df_generic_she_generic_he.drop_duplicates()
len(df_cleaned)

246